In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16, InceptionV3, MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping


In [3]:
class BirdClassifier:
    def __init__(self,  model_name, root='aml-2024-feather-in-focus/', epochs=10, num_classes=200):
        print('{}\nModel: {}\n'.format('-'*75, model_name))

        self.model_name = model_name
        self.root = root
        self.epochs = epochs
        self.num_classes = num_classes

        if self.model_name == 'InceptionV3':
            self.target_size = (299, 299)
        else:
            self.target_size = (224, 224)

        self.create_train_val_generators()
        self.create_kaggle_test_generator()
        self.setup_model()

        print('Classifier model setup complete!\n')

    def create_train_val_generators(self):

        # load labels and corresponding bird species names
        data = np.load(self.root + 'class_names.npy', allow_pickle=True)
        data = dict(data.item())    
        class_names = pd.DataFrame(data.items(), columns=['name', 'label'])

        # remove first 4 characters for every class and  replace underscores with spaces
        class_names['name'] = class_names['name'].apply(lambda x: x[4:]).apply(lambda x: x.replace('_', ' '))

        # load the paths of the bird images test set
        birds_df = pd.read_csv(self.root + 'train_images.csv')
        birds_df['image_path'] = birds_df['image_path'].apply(lambda x: self.root + 'train_images' + x)   

        # merge the labels and image paths with the species names
        birds_df = birds_df.merge(class_names, left_on='label', right_on='label')

        train_df, val_df = train_test_split(birds_df, test_size=0.2,
                                    shuffle=True,random_state=42, stratify=birds_df['name']
                                )
        
        train_datagen = ImageDataGenerator(rescale=1./255, 
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')


        print("Creating train and validation generators:")

        train_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            x_col='image_path',
            y_col='name',  
            target_size=self.target_size,
            batch_size=64,
            class_mode='sparse',
            color_mode='rgb',
            shuffle=True,
            seed=42
        )

        validation_datagen = ImageDataGenerator(rescale=1./255)
        validation_generator = validation_datagen.flow_from_dataframe(
            dataframe=val_df,
            x_col='image_path',
            y_col='name', 
            target_size=self.target_size, 
            batch_size=64,
            class_mode='sparse',
            color_mode='rgb',
            shuffle=True,
            seed=42
        )
                    
        # used for remapping predictions to bird species names
        indices = train_generator.class_indices
        self.indices_inv = {v: k for k, v in indices.items()}
        self.birds_df = birds_df
        self.class_names = class_names
        self.label_mapping = dict(zip(class_names['name'], class_names['label']))

        self.train_generator = train_generator
        self.validation_generator = validation_generator
        print()


    def create_kaggle_test_generator(self,):
        
        paths_df = pd.read_csv(self.root + 'test_images_path.csv')
        paths_df['image_path'] = paths_df['image_path'].apply(lambda x: self.root+"test_images" + x)

        print("Creating test generator:")
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_dataframe(
            dataframe=paths_df,
            x_col='image_path',
            y_col= None,
            target_size=self.target_size,  
            batch_size=64,
            class_mode=None,
            color_mode='rgb',
            shuffle=False,
            seed=42,
        )
        print()
        self.test_generator = test_generator


    def setup_model(self):
        if self.model_name == 'ResNet50':
            base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        elif self.model_name == 'EfficientNetB0':
            base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        elif self.model_name == 'VGG16':
            base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        elif self.model_name == 'InceptionV3':
            base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
        elif self.model_name == 'MobileNetV2':
            base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        else:
            raise ValueError("Model name not recognized")
        
        base_model.trainable = False

        self.model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(512, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(self.num_classes, activation='softmax') 
        ])
        
        self.model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    def save_history(self, filename_suffix = ''):
        if not filename_suffix == '':
            filename_suffix = '_' + filename_suffix

        history_df = pd.DataFrame(self.history.history)
        history_df.to_csv('saved_models/{}{}_history.csv'.format(self.model_name, filename_suffix), index=False)
        print('History saved as: history/{}{}_history.csv\n'.format(self.model_name, filename_suffix))

    def train_model(self, completion_sound=False):

                
        early_stopping = EarlyStopping(
            monitor='accuracy',  
            patience=5,         
            restore_best_weights=True  
        )

        print('Training model for {} epochs...'.format(self.epochs))
        self.history = self.model.fit(self.train_generator, epochs=self.epochs, validation_data=self.validation_generator, callbacks=[early_stopping])

        print('Training complete!\n')
        if completion_sound:
            import winsound
            for _ in range(5):
                winsound.Beep(800, 500)


    def save_model(self, filename_suffix = ''): 
        
        self.save_history(filename_suffix)

        if not filename_suffix == '':
            filename_suffix = '_' + filename_suffix
    
        self.model.save('saved_models/{}{}'.format(self.model_name, filename_suffix))
        self.model.save_weights('saved_models/{}{}_weights.h5'.format(self.model_name, filename_suffix))
        print('Model saved as: saved_models/{}{}\n'.format(self.model_name, filename_suffix))

    def load_model(self, filename_suffix = ''):
        
        if not filename_suffix == '':
            filename_suffix = '_' + filename_suffix

        self.model = load_model('saved_models/{}{}'.format(self.model_name, filename_suffix))
        self.model.load_weights('saved_models/{}{}_weights.h5'.format(self.model_name, filename_suffix))
        print('Model loaded from: saved_models/{}{}\n'.format(self.model_name, filename_suffix))

    def create_kaggle_submission(self, filename_suffix = ''):

        print('Predicting results for test images...')
        # predict the test images through the model
        pred = self.model.predict(self.test_generator, steps=len(self.test_generator), verbose=1)
        pred = np.argmax(pred, axis=1)
        print('Predictions complete!')

        # maps prediction results to bird species names
        pred = [self.indices_inv[i] for i in pred]

        # maps bird species names to numerical labels
        pred= [self.label_mapping[species] for species in pred]

        # result dataframe
        df_result = pd.DataFrame({'image_path': self.test_generator.filenames, 'label': pred})
        
        # column id only last path
        df_result['image_path'] = df_result['image_path'].apply(lambda x: x.split('/')[-1][:-4]).astype(int)

        # for conversion between filenames to kaggle id
        paths_df = pd.read_csv(self.root + 'test_images_path.csv')
        paths_df['image_path'] = paths_df['image_path'].apply(lambda x: x.split('/')[-1][:-4]).astype(int)
        paths_df = paths_df.drop(columns=['label'])

        # add only id column from paths_df to df_result, merge on image_path column
        df_result = df_result.merge(paths_df, left_on='image_path', right_on='image_path')

        #remove image_path and move colum id to last position
        df_result = df_result.drop(columns=['image_path'])
        df_result = df_result[['id', 'label']]

        # format column to int
        df_result['label'] = df_result['label'].astype(int)
        
        # sort by id
        df_result = df_result.sort_values(by='id')
        # print(df_result.head())
        # print(np.unique(df_result['label']))	

        if not filename_suffix == '':
            filename_suffix = '_' + filename_suffix
        
        df_result.to_csv('submissions/submission_{}{}.csv'.format(self.model_name, filename_suffix), index=False)

        print('Submission file created: submissions/submission_{}{}.csv\n'.format(self.model_name, filename_suffix))



In [ ]:
# model_name = 'ResNet50' 
# model_name = 'EfficientNetB0'
# model_name = 'VGG16'
# model_name = 'InceptionV3'
# model_name = 'MobileNetV2'


# model_names = ['MobileNetV2', 'ResNet50', 'EfficientNetB0', 'VGG16', 'InceptionV3']

model_names = ['MobileNetV2', 'ResNet50', 'EfficientNetB0', 'InceptionV3']


for model_name in model_names:
    classifier = BirdClassifier(model_name=model_name, epochs=50)
    classifier.train_model(completion_sound=False)
    classifier.create_kaggle_submission("50epochs")
    try:
        classifier.save_model("50epochs")
    except:pass